In [1]:
import pandas as pd
import numpy as np
from nsepython import equity_history
from datetime import datetime, timedelta

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
%run "./technical_analysis_self.ipynb"

In [3]:
script_name = 'infy'
series = 'EQ'
end_date = datetime.now().date()
start_date = end_date - timedelta(days = 400)

In [4]:
df = DataLoader.load_data(
    script_name, 
    start_date.strftime("%d-%m-%Y"), 
    end_date.strftime("%d-%m-%Y"), 
    series).sort_values('date').drop_duplicates()

c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsepython\rahu.py:544: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  total=total.append(equity_history_virgin(symbol,series,start_date,temp_date))
c:\users\ragho\appdata\local\programs\python\python38\lib\site-packages\nsep

In [5]:
for fn in [load_volume_sma, load_emas, load_macd, load_bollinger_bands, load_rsi, load_ATR]:
    df = fn(df)
    
df = load_ADX(df, 14)
df = load_aroon(df, 14)
df['pct_change'] = df['close'].diff(1) * 100 / df['prev_close']

In [6]:
df.tail()

,date,symbol,high,low,open,close,ltp,prev_close,volume,high_52w,...,plus_DI,minus_DI,DX,ADX,period,days_since_period_high,days_since_period_low,aroon_up,aroon_down,pct_change
22,2022-09-22,INFY,1383.95,1360.0,1366.35,1367.95,1368.00,1377.25,9168498,1953.9,...,18.258775,42.318539,39.717450,24.876514,14,7.0,0.0,50.000000,100.000000,-0.675259
23,2022-09-23,INFY,1394.80,1362.0,1367.50,1365.45,1367.95,1367.95,10426200,1953.9,...,19.276222,39.413312,34.311211,25.550421,14,8.0,1.0,42.857143,92.857143,-0.182755
24,2022-09-26,INFY,1399.45,1355.0,1360.00,1380.25,1383.00,1365.45,9414587,1953.9,...,17.520784,37.258173,36.030969,26.299032,14,9.0,0.0,35.714286,100.000000,1.083892
25,2022-09-27,INFY,1406.50,1378.0,1393.45,1393.55,1398.25,1380.25,8561254,1953.9,...,17.947694,35.053937,32.274936,26.725882,14,10.0,1.0,28.571429,92.857143,0.963594
26,2022-09-28,INFY,1405.85,1377.0,1379.00,1394.70,1394.45,1393.55,7022098,1953.9,...,16.860299,33.140134,32.559388,27.142561,14,11.0,2.0,21.428571,85.714286,0.082523


In [7]:
initial_columns = list(df.columns)

df["next_day_open"] = df["open"].shift(-1)

# utils

In [8]:
class SessionHigh:
    def __init__(self):
        self.current_session_high, self.buy_flag = 0, False

    def get_session_high(self, row):
#         global current_session_high, buy_flag, current_session_highs
        if row['indicator_signal'] == 'buy':
            self.current_session_high, self.buy_flag = 0, True
            return 0
        elif row['indicator_signal'] == 'sell':
            self.current_session_high, self.buy_flag = 0, False
            return self.current_session_high
        elif self.buy_flag:
            self.current_session_high = self.current_session_high if self.current_session_high > row['high'] else row['high']
            return self.current_session_high
        return None
    
    def reset(self):
        self.current_session_high = 0
        self.buy_flag = False

In [9]:
def get_stoploss_columns(df, stoploss_pct=2):
    df[f'sl_prev_day'] = df['moving_session_high'].shift(1) * (100 - stoploss_pct) / 100
    df[f'sl_curr_day'] = df['moving_session_high'] * (100 - stoploss_pct) / 100
    df['sl_prev_day_triggered'] = df[f'sl_prev_day'].between(df['low'], df['high'])
    df['sl_curr_day_triggered'] = df[f'sl_curr_day'].between(df['low'], df['high'])
    return df

def get_returns_df(df, sl_column='sl_prev_day_triggered'):
    returns_df = df[(df['indicator_signal'] == 'buy') | (df[sl_column])].copy()

    returns_df[sl_column] = np.where(
        returns_df[sl_column].shift(1) == True,
        False,
        returns_df[sl_column]
    )

    returns_df = returns_df[(returns_df['indicator_signal'] == 'buy') | (returns_df[sl_column])].copy()
    returns_df['return_pct'] = (returns_df[sl_column.replace('_triggered', '')] - returns_df['next_day_open'].shift(1)) * 100 / returns_df['next_day_open'].shift(1)
    
    return returns_df

# Moving average crossovers

### 9d - 21d crossover

In [10]:
required_columns = ["date", "symbol", "high", "low", "open", "close", "prev_close", "next_day_open", "pct_change"]

In [103]:
df_9_21_co = df.copy()

df_9_21_co['indicator_signal'] = np.where(
    np.logical_or(
        np.logical_and(
            df_9_21_co['ema_5d'] > df_9_21_co['ema_9d'],
            df_9_21_co['ema_5d'].shift(1) < df_9_21_co['ema_9d'].shift(1)
        ),
        np.logical_and(
            df_9_21_co['ema_5d'] < df_9_21_co['ema_9d'],
            df_9_21_co['ema_5d'].shift(1) > df_9_21_co['ema_9d'].shift(1)
        )
    ),
    df_9_21_co[['ema_5d', 'ema_9d']].idxmax(axis=1).replace(['ema_5d', 'ema_9d'], ['buy', 'sell']),
    None
)

df_9_21_co = df_9_21_co[required_columns + ["indicator_signal"]].set_index('date')

df_9_21_co['moving_session_high'] = df_9_21_co.apply(SessionHigh().get_session_high, axis=1)
df_9_21_co = get_stoploss_columns(df_9_21_co, stoploss_pct=3)
returns_df = get_returns_df(df_9_21_co)

In [104]:
returns_df[['high', 'low', 'open', 'close', 'next_day_open', 'indicator_signal', 'sl_prev_day', 'sl_prev_day_triggered', 'return_pct']]

,high,low,open,close,next_day_open,indicator_signal,sl_prev_day,sl_prev_day_triggered,return_pct
date,,,,,,,,,
2021-08-25,1746.90,1718.00,1718.00,1735.55,1734.00,buy,NaN,False,NaN
2021-09-06,1732.00,1699.35,1705.15,1730.40,1730.00,buy,NaN,False,NaN
2021-09-21,1726.75,1684.65,1700.00,1718.45,1717.00,buy,NaN,False,NaN
2021-09-27,1769.00,1718.10,1768.75,1721.15,1710.00,None,1734.3600,True,1.011066
2021-10-08,1731.50,1698.40,1705.25,1723.85,1682.00,buy,NaN,False,NaN
2021-10-12,1689.00,1662.00,1677.00,1685.15,1695.95,sell,1678.1000,True,-0.231867
2021-10-13,1713.65,1680.05,1695.95,1709.20,1754.00,buy,0.0000,False,-100.000000
2021-10-21,1814.00,1746.85,1814.00,1753.65,1749.90,None,1792.5600,True,2.198404
2021-11-08,1741.00,1703.00,1730.00,1737.50,1734.75,buy,NaN,False,NaN


# MACD - Signal line Crossover

In [105]:
df_macd = df[required_columns + ['macd', 'macd_9d_signal']].copy().set_index('date')

df_macd['indicator_signal'] = np.where(
    np.logical_or(
        np.logical_and(
            df_macd['macd'] > df_macd['macd_9d_signal'],
            df_macd['macd'].shift(1) < df_macd['macd_9d_signal'].shift(1)
        ),
        np.logical_and(
            df_macd['macd'] < df_macd['macd_9d_signal'],
            df_macd['macd'].shift(1) > df_macd['macd_9d_signal'].shift(1)
        )
    ),
    df_macd[['macd', 'macd_9d_signal']].idxmax(axis=1).replace({"macd": "buy", "macd_9d_signal": "sell"}),
    None
)

df_macd['moving_session_high'] = df_macd.apply(SessionHigh().get_session_high, axis=1)
df_macd = get_stoploss_columns(df_macd, stoploss_pct=3)
macd_returns_df = get_returns_df(df_macd)

macd_returns_df

,symbol,high,low,open,close,prev_close,next_day_open,pct_change,macd,macd_9d_signal,indicator_signal,moving_session_high,sl_prev_day,sl_curr_day,sl_prev_day_triggered,sl_curr_day_triggered,return_pct
date,,,,,,,,,,,,,,,,,
2021-08-25,INFY,1746.90,1718.00,1718.00,1735.55,1720.85,1734.00,0.854229,-0.055875,-0.153992,buy,0.00,NaN,0.0000,False,False,NaN
2021-09-06,INFY,1732.00,1699.35,1705.15,1730.40,1700.65,1730.00,1.749331,-1.672635,-2.042891,buy,0.00,NaN,0.0000,False,False,NaN
2021-09-16,INFY,1719.25,1691.45,1715.20,1702.25,1711.45,1697.00,-0.537556,-2.202539,-2.402967,buy,0.00,NaN,0.0000,False,False,NaN
2021-09-21,INFY,1726.75,1684.65,1700.00,1718.45,1687.85,1717.00,1.812957,-1.457864,-2.389057,buy,0.00,NaN,0.0000,False,False,NaN
2021-09-27,INFY,1769.00,1718.10,1768.75,1721.15,1763.85,1710.00,-2.420841,5.639818,-0.128320,None,1788.00,1734.3600,1734.3600,True,True,1.011066
2021-10-13,INFY,1713.65,1680.05,1695.95,1709.20,1685.15,1754.00,1.427173,-2.265040,-2.484357,buy,0.00,NaN,0.0000,False,False,NaN
2021-10-21,INFY,1814.00,1746.85,1814.00,1753.65,1802.35,1749.90,-2.702028,18.162755,3.689869,None,1848.00,1792.5600,1792.5600,True,True,2.198404
2021-11-10,INFY,1750.95,1721.95,1726.00,1740.15,1742.50,1732.95,-0.134864,3.186418,2.231153,buy,0.00,NaN,0.0000,False,False,NaN
2021-11-22,INFY,1785.40,1751.15,1784.00,1759.40,1779.40,1735.10,-1.123974,16.312565,8.186052,None,1808.95,1754.6815,1754.6815,True,True,1.254018


# ADX Based Signals

In [106]:
df_adx = df[required_columns + ["plus_DI", "minus_DI", "DX", "ADX"]].copy()

df_adx['indicator_signal'] = np.where(
    np.logical_and(
        df_adx["ADX"] >= 18,
        np.logical_or(
            np.logical_and(df_adx["plus_DI"] > df_adx["minus_DI"], df_adx["plus_DI"].shift(1) < df_adx["minus_DI"].shift(1)),
            np.logical_and(df_adx["plus_DI"] < df_adx["minus_DI"], df_adx["plus_DI"].shift(1) > df_adx["minus_DI"].shift(1)),
        )
    ),
    df_adx[["plus_DI", "minus_DI"]].idxmax(axis=1).replace({'plus_DI': 'buy', 'minus_DI': 'sell'}),
    None
)

df_adx['moving_session_high'] = df_adx.apply(SessionHigh().get_session_high, axis=1)
df_adx = get_stoploss_columns(df_adx, stoploss_pct=2)
adx_returns_df = get_returns_df(df_adx)
adx_returns_df

,date,symbol,high,low,open,close,prev_close,next_day_open,pct_change,plus_DI,minus_DI,DX,ADX,indicator_signal,moving_session_high,sl_prev_day,sl_curr_day,sl_prev_day_triggered,sl_curr_day_triggered,return_pct
270,2021-09-21,INFY,1726.75,1684.65,1700.00,1718.45,1687.85,1717.0,1.812957,17.565568,16.888521,1.965069,39.803286,buy,0.0,NaN,0.000,False,False,NaN
274,2021-09-27,INFY,1769.00,1718.10,1768.75,1721.15,1763.85,1710.0,-2.420841,25.773867,19.842824,13.001914,34.083569,None,1788.0,1752.240,1752.240,True,True,2.052417
228,2021-10-08,INFY,1731.50,1698.40,1705.25,1723.85,1691.00,1682.0,1.942637,23.656096,18.589128,11.994180,22.337607,buy,0.0,NaN,0.000,False,False,NaN
231,2021-10-13,INFY,1713.65,1680.05,1695.95,1709.20,1685.15,1754.0,1.427173,22.967173,20.674325,5.253825,18.993957,buy,0.0,NaN,0.000,False,False,NaN
233,2021-10-18,INFY,1799.00,1740.00,1740.00,1792.15,1715.75,1814.0,4.452863,30.639830,14.680703,35.213901,21.075690,None,1799.0,1747.928,1763.020,True,True,-0.346180
180,2022-01-07,INFY,1836.00,1806.80,1815.45,1814.30,1817.80,1815.0,-0.192540,25.617955,24.785447,1.651690,22.996133,buy,0.0,0.000,0.000,False,False,-100.000000
187,2022-01-18,INFY,1945.00,1911.45,1922.10,1920.75,1939.50,1916.0,-0.966744,32.938964,18.141613,28.968644,24.502190,None,1953.9,1914.822,1914.822,True,True,5.499835
163,2022-03-08,INFY,1800.00,1718.45,1718.45,1778.20,1739.85,1791.0,2.204213,24.330270,22.910304,3.005820,18.530152,buy,0.0,NaN,0.000,False,False,NaN
115,2022-03-15,INFY,1888.35,1830.00,1888.35,1839.30,1890.70,1865.0,-2.718570,30.496789,15.814567,31.703285,19.806972,None,1895.0,1857.100,1857.100,True,True,3.690676
132,2022-04-08,INFY,1838.00,1808.00,1823.00,1814.60,1811.00,1814.0,0.198785,19.751037,19.446080,0.778008,26.088840,buy,0.0,0.000,0.000,False,False,-100.000000


# Aroon

In [107]:
aroon_df = df[required_columns + ["aroon_up", "aroon_down"]].copy()

In [108]:
aroon_df['indicator_signal'] = np.where(
    np.logical_or(
        np.logical_and(aroon_df["aroon_up"] > aroon_df["aroon_down"], aroon_df["aroon_up"].shift(1) < aroon_df["aroon_down"].shift(1)),
        np.logical_and(aroon_df["aroon_up"] < aroon_df["aroon_down"], aroon_df["aroon_up"].shift(1) > aroon_df["aroon_down"].shift(1)),
    ),
    aroon_df[["aroon_up", "aroon_down"]].idxmax(axis=1).replace({'aroon_up': 'buy', 'aroon_down': 'sell'}),
    None,
)

aroon_df['moving_session_high'] = aroon_df.apply(SessionHigh().get_session_high, axis=1)
aroon_df = get_stoploss_columns(aroon_df, stoploss_pct=2.5)
aroon_returns_df = get_returns_df(aroon_df)
aroon_returns_df

,date,symbol,high,low,open,close,prev_close,next_day_open,pct_change,aroon_up,aroon_down,indicator_signal,moving_session_high,sl_prev_day,sl_curr_day,sl_prev_day_triggered,sl_curr_day_triggered,return_pct
267,2021-09-16,INFY,1719.25,1691.45,1715.20,1702.25,1711.45,1697.00,-0.537556,57.142857,35.714286,buy,0.00,NaN,0.00000,False,False,NaN
274,2021-09-27,INFY,1769.00,1718.10,1768.75,1721.15,1763.85,1710.00,-2.420841,92.857143,28.571429,None,1788.00,1743.30000,1743.30000,True,True,2.728344
232,2021-10-14,INFY,1783.60,1698.20,1754.00,1715.75,1709.20,1740.00,0.383220,100.000000,21.428571,buy,0.00,NaN,0.00000,False,False,NaN
236,2021-10-21,INFY,1814.00,1746.85,1814.00,1753.65,1802.35,1749.90,-2.702028,92.857143,7.142857,None,1848.00,1801.80000,1801.80000,True,True,3.551724
197,2021-11-12,INFY,1782.85,1738.05,1743.00,1780.20,1733.30,1782.10,2.705821,100.000000,35.714286,buy,0.00,NaN,0.00000,False,False,NaN
201,2021-11-18,INFY,1789.45,1759.00,1781.60,1779.40,1787.45,1784.00,-0.450362,85.714286,7.142857,None,1808.95,1763.72625,1763.72625,True,True,-1.031017
215,2021-12-09,INFY,1769.40,1742.00,1759.00,1763.45,1753.35,1760.00,0.576040,71.428571,42.857143,buy,0.00,NaN,0.00000,False,False,NaN
218,2021-12-14,INFY,1763.00,1710.35,1730.90,1752.15,1744.65,1745.00,0.429886,50.000000,21.428571,None,1771.35,1727.06625,1727.06625,True,True,-1.871236
185,2022-01-14,INFY,1933.00,1881.00,1882.00,1929.35,1896.80,1938.55,1.716048,100.000000,57.142857,buy,0.00,NaN,0.00000,False,False,NaN
188,2022-01-19,INFY,1916.00,1862.85,1916.00,1867.05,1920.75,1844.00,-2.795783,85.714286,35.714286,None,1953.90,1905.05250,1905.05250,True,True,-1.727967


# Candlestick pattern based signals

In [109]:
# df_candles = df[required_columns + ["ADX"]].copy()
# df_candles["candle_color"] = df_candles[['close', 'prev_close']].idxmax(axis=1).replace({'close': 'green', 'prev_close': 'red'})

# for fn in [identify_marubozus, identify_dojis, identify_paper_umbrellas, identify_shooting_stars]:
#     df_candles = fn(df_candles)